## 导包

In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Lambda, Compose

In [2]:
from config import Config
from kits.models import *
from kits.trainer import BaseModelTrainer, parse_data
# print(torch.cuda.is_available())
# print('1')

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


## 数据预处理

In [3]:
import pandas as pd
# 文件路径
data_path = '/home/data/sunyingjie/zeng_appdata/'
mini_data_path = '/home/data/sunyingjie/zeng_appdata/mini_data/'
# 文件名
train_sample = 'train_sample.csv'
valid_sample = 'valid_sample.csv'
test_sample = 'test_sample.csv'

train_data = pd.read_csv(data_path + train_sample, encoding='utf-8-sig', sep='|')
valid_data = pd.read_csv(data_path + valid_sample, encoding='utf-8-sig', sep='|')
test_data = pd.read_csv(data_path + test_sample, encoding='utf-8-sig', sep='|')
train_data.head()

,uid,new_soft_list,new_time_list,new_sep_list,age,gender
0,141879,"183, 69, 111, 127, 27, 153, 19, 5, 434, 302, 1...","1917, 1917, 1917, 1917, 1917, 1917, 1917, 1917...","1627, 1627, 1627, 1627, 1627, 1627, 1627, 1627...",2,0
1,98074,"471, 763, 8, 2056, 62, 15, 417, 273, 13, 255, ...","2245, 2245, 2245, 2245, 2245, 2245, 2245, 2245...","1286, 1286, 1286, 1286, 1286, 1286, 1286, 1286...",1,1
2,102602,"6, 60, 15, 5, 56, 9, 170, 1, 47, 2045, 299, 65...","2436, 2436, 2436, 2436, 2436, 2436, 2436, 2436...","1098, 1098, 1098, 1098, 1098, 1098, 1098, 1098...",2,0
3,38051,"70, 16, 231, 93, 15, 1395, 47, 5, 125, 30, 40,...","2307, 2307, 2307, 2307, 2307, 2307, 2307, 2307...","1230, 1230, 1230, 1230, 1230, 1230, 1230, 1230...",3,0
4,40634,"0, 17, 98, 5, 27, 186, 1104, 13628, 49, 45, 43...","1697, 1697, 1697, 1697, 1697, 1697, 1697, 1697...","1838, 1838, 1838, 1838, 1838, 1838, 1838, 1838...",1,1


In [4]:
num_row = train_data.shape[0]/10

In [5]:
train_data = pd.read_csv(data_path + train_sample, encoding='utf-8-sig', sep='|', nrows = train_data.shape[0]/100)
valid_data = pd.read_csv(data_path + valid_sample, encoding='utf-8-sig', sep='|', nrows = valid_data.shape[0]/100)
test_data  = pd.read_csv(data_path + test_sample, encoding='utf-8-sig', sep='|', nrows = test_data.shape[0]/100)

In [6]:
train_data.shape

(800, 6)

In [7]:
train_data.to_csv(mini_data_path + 'mini_train_data.csv', encoding='utf-8-sig', sep='|', index=False)
valid_data.to_csv(mini_data_path + 'mini_valid_data.csv', encoding='utf-8-sig', sep='|', index=False)
test_data.to_csv(mini_data_path + 'mini_test_data.csv', encoding='utf-8-sig', sep='|', index=False)

In [8]:
train_data = pd.read_csv(mini_data_path + 'mini_train_data.csv', encoding='utf-8-sig', sep='|')
train_data.head()

,uid,new_soft_list,new_time_list,new_sep_list,age,gender
0,141879,"183, 69, 111, 127, 27, 153, 19, 5, 434, 302, 1...","1917, 1917, 1917, 1917, 1917, 1917, 1917, 1917...","1627, 1627, 1627, 1627, 1627, 1627, 1627, 1627...",2,0
1,98074,"471, 763, 8, 2056, 62, 15, 417, 273, 13, 255, ...","2245, 2245, 2245, 2245, 2245, 2245, 2245, 2245...","1286, 1286, 1286, 1286, 1286, 1286, 1286, 1286...",1,1
2,102602,"6, 60, 15, 5, 56, 9, 170, 1, 47, 2045, 299, 65...","2436, 2436, 2436, 2436, 2436, 2436, 2436, 2436...","1098, 1098, 1098, 1098, 1098, 1098, 1098, 1098...",2,0
3,38051,"70, 16, 231, 93, 15, 1395, 47, 5, 125, 30, 40,...","2307, 2307, 2307, 2307, 2307, 2307, 2307, 2307...","1230, 1230, 1230, 1230, 1230, 1230, 1230, 1230...",3,0
4,40634,"0, 17, 98, 5, 27, 186, 1104, 13628, 49, 45, 43...","1697, 1697, 1697, 1697, 1697, 1697, 1697, 1697...","1838, 1838, 1838, 1838, 1838, 1838, 1838, 1838...",1,1


In [9]:
train_data.shape

(800, 6)

In [10]:
train_data.dtypes

uid               int64
new_soft_list    object
new_time_list    object
new_sep_list     object
age               int64
gender            int64
dtype: object